In [1]:
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 
import datetime
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
today=str(datetime.datetime.today().strftime('%d_%m_%Y'))

In [3]:
INC = pd.read_excel(r'C:\NotBackedUp\SOE\source_files\INC\incident.xlsx')
model1map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\model1_mapping.csv')
siam_map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\SIAM_ServiceMapping.csv')

In [4]:
INC = INC.rename(columns={'Name':'business_service_name'})

In [5]:
INC['business_service_name'] = INC['business_service_name'].str.strip('[PROD]')
INC['business_service_name'] = INC['business_service_name'].str.strip()

In [6]:
days_before = (date.today()-timedelta(days=30))

In [7]:
INC['Opened'] = INC['Opened'].fillna(0)
INC['Opened'] = pd.to_datetime(INC['Opened'],format='%Y-%m-%d')
INC['Opened'] = INC['Opened'].apply(lambda x: x.strftime('%Y-%m-%d'))
INC['Opened']= pd.to_datetime(INC['Opened'],format='%Y-%m-%d')

In [8]:
INC['Created'] = INC['Created'].fillna(0)
INC['Created'] = pd.to_datetime(INC['Created'],format='%Y-%m-%d')
INC['Created'] = INC['Created'].apply(lambda x: x.strftime('%Y-%m-%d'))
INC['Created']= pd.to_datetime(INC['Created'],format='%Y-%m-%d')

In [9]:
INC = INC[['Number','Created','State','business_service_name','Opened','Priority','Severity']]

In [10]:
INC=INC[(INC['State'].isin(['New','In Progress','On Hold']))]

In [11]:
INC['Days'] = INC['Opened'] - INC['Created']

In [12]:
Model1AGEINC = INC.merge(model1map,how='left',left_on=['business_service_name'],right_on=['Name'])
SIAMAGEINC = INC.merge(siam_map,how='left',left_on=['business_service_name'],right_on=['Name'])

In [13]:
Model1AGEINC = Model1AGEINC[['Domain','Tribe_TechArea','Number','Created','State','business_service_name','Opened','Priority','Severity','Days','Name']]
SIAMAGEINC = SIAMAGEINC[['Domain','Tribe_TechArea','Number','Created','State','business_service_name','Opened','Priority','Severity','Days','Name']]

In [14]:
Model1AGEINC['Name'] = Model1AGEINC['Name'].fillna(0)
SIAMAGEINC['Name'] = SIAMAGEINC['Name'].fillna(0)

In [15]:
Model1AGEINC = Model1AGEINC[Model1AGEINC['Name'] != 0]
SIAMAGEINC = SIAMAGEINC[SIAMAGEINC['Name'] != 0]

In [16]:
Model1AGEINC=Model1AGEINC[(Model1AGEINC['Days']=='30 days')]
SIAMAGEINC=SIAMAGEINC[(SIAMAGEINC['Days']=='30 days')]

In [18]:
SIAMAGEINC.to_csv(r'C:\NotBackedUp\SOE\SIAM\CSV\AGED_INC_'+today+'.csv',index=None)
Model1AGEINC.to_csv(r'C:\NotBackedUp\SOE\Model1\CSV\AGED_INC_'+today+'.csv',index=None)